In [40]:
import torch
import numpy as np
import yaml
import matplotlib.pyplot as plt
import cudaq
from models import FFNN
from moleculars import get_pyscf_results, MOLECULE_DATA
from vmc_cal import *
from vqe_details import *
from tqdm import trange


def load_config(path="config.yaml"):
    with open(path, 'r') as file:
        return yaml.safe_load(file)


def plot_results(results, seps, config):
    method_name = "FFNN+VQE"
    molecule_name = config['molecule_name']
    basis = MOLECULE_DATA[molecule_name]['basis']
    plt.figure(figsize=(12, 8))
    results_VMC = results.pop(method_name)
    for method, values in results.items():
        plt.plot(seps, values, linestyle='--', label=method)
    energies = [res[0] for res in results_VMC]
    errors = [res[1] for res in results_VMC]
    plt.errorbar(seps, energies, yerr=errors, marker='o', linestyle='-', label=method_name, capsize=5)
    plt.xlabel("Separation (Å)"), plt.ylabel("Energy (Hartree)")
    plt.title(f"{molecule_name} Potential Energy Dissociation Curve ({basis})")
    plt.legend(), plt.grid(True, which='both', linestyles='--', linewidth=0.5)
    plt.tight_layout(), plt.show()


def adjust_lr(initial_lr, epoch, schedule_type, T_max, decay_rate=0.98):
    if schedule_type == "cosine":
        return initial_lr * 0.5 * (1 + np.cos(np.pi * epoch / T_max))
    elif schedule_type == "exp":
        return initial_lr * (decay_rate ** epoch)
    else:
        return initial_lr
    

def run_nqs_sr_update(
    nqs_model, vmc_params, qham_of, n_orbitals, device, 
    best_energy, best_state_dict, patience, start_epoch=0, extra_epochs=10):
    """
    Perform additional local SR updates with early stopping.
    """
    no_improve_count = 0
    for ep in range(extra_epochs):
        lr = adjust_lr(vmc_params['learning_rate'], start_epoch + ep, 
                       schedule_type="cosine", T_max=start_epoch + extra_epochs)

        samples = efficient_parallel_sampler(
            nqs_model, 
            vmc_params['n_samples'] // vmc_params['n_chains'], 
            vmc_params['n_chains'], 
            n_orbitals,
            vmc_params['burn_in_steps'], 
            vmc_params['step_intervals'], 
            device=device
        )

        stochastic_reconfiguration_update(
            nqs_model, samples, qham_of,
            lr=lr,
            reg=vmc_params['sr_regularization'],
            device=device
        )

        eval_local_energies = local_energy_batch(nqs_model, samples, qham_of, device)
        eval_mean = eval_local_energies.mean().item()
        eval_std = eval_local_energies.std().item() / np.sqrt(len(eval_local_energies))

        if ep % 10 == 0:  
            print(f"[SR Update Epoch {ep + 1}] "
                  f"Eval Energy: {eval_mean:.6f} ± {eval_std:.6f} Ha")

        if eval_mean < best_energy:
            best_energy = eval_mean
            best_state_dict = {k: v.clone().detach().cpu() for k, v in nqs_model.state_dict().items()}
            no_improve_count = 0
        else:
            no_improve_count += 1

        if no_improve_count >= patience:
            print(f"[Early Stopping] No improvement for {patience} epochs in SR update.")
            break

    if best_state_dict is not None:
        nqs_model.load_state_dict(best_state_dict)
        print(f"[Step 1b] Restored best SR-updated parameters with energy = {best_energy:.6f} Ha")

    print("[Step 1b] Extra SR update finished.")
    return best_energy, best_state_dict

In [41]:
if __name__ == '__main__':
    config = load_config()
    hybrid_params = config['hybrid_params']
    molecule_choice = config['molecule_name']
    vmc_params = config['vmc_params']
    ffnn_params = config['ffnn_params']
    hew_params = config['hew_params']

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # cuda:0, cuda:1
    cudaq.set_target("nvidia")
    print(f"Using PyTorch device: {device}, CUDA-Q target: {cudaq.get_target().name}")

    geom = MOLECULE_DATA[molecule_choice]['geometry']
    curve_config = config['dissociation_curve']
    seps = np.linspace(curve_config['start_separation'], curve_config['end_separation'], curve_config['points'])
    base_dist = np.linalg.norm(np.array(geom[0][1]) - np.array(geom[1][1]))
    scales = seps / base_dist if base_dist > 0 else np.ones_like(seps)

    results = {'HF': [], 'FCI': [], 'CCSD': [], 'CCSD(T)': [], "FFNN+VQE": []}
    
    for i, scale in enumerate(scales):
        label = f"{seps[i]:.3f} Å"
        print(f"\n--- Calculating for {molecule_choice} @ {label} ---")

        (
        mol_pyscf, hf_e, fci_e, ccsd_e, ccsd_t_e, qham_of,
        hcore, eri, nuclear_repulsion_energy, n_orb, n_elec) = get_pyscf_results(molecule_choice, scale)

        mol_geom_for_cudaq = [(str(atom), tuple(pos)) for atom, pos in mol_pyscf.atom]
        molecule_ham, data = cudaq.chemistry.create_molecular_hamiltonian(mol_geom_for_cudaq, MOLECULE_DATA[molecule_choice]['basis'])
        results['HF'].append(hf_e); results['FCI'].append(fci_e); results['CCSD'].append(ccsd_e); results['CCSD(T)'].append(ccsd_t_e)

        print(f"  Hartree-Fock: {hf_e:.6f} Ha")
        print(f"  FCI:          {fci_e:.6f} Ha")
        print(f"  CCSD:         {ccsd_e:.6f} Ha")
        print(f"  CCSD(T):      {ccsd_t_e:.6f} Ha")

        n_orbitals = mol_pyscf.nao_nr() * 2
        n_hidden = int(n_orbitals * ffnn_params['alpha'])
        nqs_model = FFNN(n_orbitals, n_hidden, ffnn_params['n_layers'], device=device)

        # --- HYBRID ALGORITHM WITH FEEDBACK LOOP ---
        
        # Step 1: Initial NQS Pre-training with early stopping
        print(f"[Step 1] Starting NQS pre-training for {hybrid_params['nqs_pretrain_epochs']} epochs...")

        best_energy = float("inf")
        best_state_dict = None
        patience = 100   # stop if no improvement for 100 epochs
        no_improve_count = 0

        for ep in trange(hybrid_params['nqs_pretrain_epochs'], desc="NQS Pre-training"):
            
            lr = adjust_lr(vmc_params['learning_rate'], ep, schedule_type="cosine", T_max=hybrid_params['nqs_pretrain_epochs'])

            samples = efficient_parallel_sampler(
                nqs_model, 
                vmc_params['n_samples'] // vmc_params['n_chains'], 
                vmc_params['n_chains'], 
                n_orbitals,
                vmc_params['burn_in_steps'], 
                vmc_params['step_intervals'], 
                device=device
            )

            stochastic_reconfiguration_update(
                nqs_model, samples, qham_of,
                lr=lr,
                reg=vmc_params['sr_regularization'],
                device=device
            )

            eval_local_energies = local_energy_batch(nqs_model, samples, qham_of, device)
            eval_mean = eval_local_energies.mean().item()
            eval_std = eval_local_energies.std().item() / np.sqrt(len(eval_local_energies))

            if (ep + 1) % 10 == 0:
                print(f"[Epoch {ep + 1}] Eval Energy: {eval_mean:.6f} ± {eval_std:.6f} Ha")

            if eval_mean < best_energy:
                best_energy = eval_mean
                best_state_dict = {k: v.clone().detach().cpu() for k, v in nqs_model.state_dict().items()}
                no_improve_count = 0
            else:
                no_improve_count += 1

            if no_improve_count >= patience:
                print(f"[Early Stopping] No improvement for {patience} epochs. Stopping pre-training.")
                break

        if best_state_dict is not None:
            nqs_model.load_state_dict(best_state_dict)
            print(f"[Step 1] Restored best model parameters with energy = {best_energy:.6f} Ha")
        
        # best_energy, best_state_dict = run_nqs_sr_update(
        #     nqs_model, vmc_params, qham_of, n_orbitals, device, 
        #     best_energy, best_state_dict, patience,
        #     start_epoch=0, extra_epochs=hybrid_params['nqs_pretrain_epochs'])
        
        print("[Step 1] NQS pre-training finished.")
        final_energy = 0.0
        
        # vqe_energy, target_state_vector = run_vqe_fine_tuning(
        #     molecule_ham, nqs_model, n_orbitals, data.n_electrons, hew_params['n_layers'],
        #     vmc_params, qham_of, hybrid_params['vqe_max_iterations'], device
        # )

Using PyTorch device: cuda, CUDA-Q target: nvidia

--- Calculating for LiH @ 0.800 Å ---
  Hartree-Fock: -7.615770 Ha
  FCI:          -7.634167 Ha
  CCSD:         -7.634161 Ha
  CCSD(T):      -7.634167 Ha
[Step 1] Starting NQS pre-training for 20 epochs...


NQS Pre-training:  50%|█████     | 10/20 [00:43<00:45,  4.55s/it]

[Epoch 10] Eval Energy: -5.273798 ± 0.050212 Ha


NQS Pre-training: 100%|██████████| 20/20 [01:38<00:00,  4.91s/it]

[Epoch 20] Eval Energy: -5.559600 ± 0.043975 Ha
[Step 1] Restored best model parameters with energy = -5.636886 Ha
[Step 1] NQS pre-training finished.


In [53]:
# --- [您要新增的程式碼] ---
print("\n[Step 2] Sampling from final trained NQS model to find configurations...")

# 進行一次大規模採樣
final_samples_tensor = efficient_parallel_sampler(
    nqs_model, 
    10,  # 提高樣本數以獲得更準確的統計
    vmc_params['n_chains'], 
    n_orbitals,
    vmc_params['burn_in_steps'], 
    vmc_params['step_intervals'], 
    device=device
)

# 將 PyTorch Tensors 轉換為可 hash 的 tuple，以便計數
samples_list = [tuple(s.cpu().numpy().astype(int)) for s in final_samples_tensor]

from collections import Counter
config_counts = Counter(samples_list)

total_samples = len(samples_list)
print(f"\nFound {len(config_counts)} unique configurations from {total_samples} samples.")
print("--- Top 5 Most Probable Configurations ---")
total_probability_check = 0
config_str_counts = Counter()
for config_tuple, count in config_counts.most_common(len(config_counts)):
    # 將 (1, 1, 0, 0) 這樣的 tuple 轉換為 |1100> 這樣的字串
    # config_str = "".join(map(str, config_tuple))
    probability = (count / total_samples) * 100

    mapped_bits = [("0" if s == -1 else "1") for s in config_tuple]
    config_str = "".join(mapped_bits)
    total_probability_check += probability
    config_str_counts[config_str] += count
    print(f"  Config: |{config_str}>,  Probability: {probability:.4f}%  (Count: {count})")
    
print("total_probability=", total_probability_check)
config_counts = dict(config_str_counts)

print("\n--- [最終的 config_str -> Count 映射字典] ---")
print(config_counts)
number_of_unique_configs = len(config_counts)
print(f"\n字典 'final_config_str_dict' 中總共有 {number_of_unique_configs} 個不重複的項目。")


[Step 2] Sampling from final trained NQS model to find configurations...

Found 1445 unique configurations from 2560 samples.
--- Top 5 Most Probable Configurations ---
  Config: |000110110111>,  Probability: 0.3516%  (Count: 9)
  Config: |001001001111>,  Probability: 0.3516%  (Count: 9)
  Config: |000111111111>,  Probability: 0.3125%  (Count: 8)
  Config: |001011110111>,  Probability: 0.3125%  (Count: 8)
  Config: |000111111101>,  Probability: 0.3125%  (Count: 8)
  Config: |001110111111>,  Probability: 0.2734%  (Count: 7)
  Config: |001111001111>,  Probability: 0.2734%  (Count: 7)
  Config: |001101101001>,  Probability: 0.2734%  (Count: 7)
  Config: |001001111111>,  Probability: 0.2734%  (Count: 7)
  Config: |001111000101>,  Probability: 0.2734%  (Count: 7)
  Config: |001011110011>,  Probability: 0.2734%  (Count: 7)
  Config: |001110011101>,  Probability: 0.2734%  (Count: 7)
  Config: |000101110111>,  Probability: 0.2734%  (Count: 7)
  Config: |001101001111>,  Probability: 0.2734%  (

In [54]:
n_elec

(2, 2)

In [55]:
import collections


# --- 2. 守恆條件 ---
EXPECTED_N_ELEC = n_elec[0]+n_elec[1]
EXPECTED_S_Z_TIMES_2 = 0  # S_z * 2 (即 N_up - N_down)

# --- 3. 篩選 ---
conserved_states = collections.Counter()
total_samples = sum(config_counts.values())

for config_str, count in config_counts.items():
    
    # 檢查 N_elec
    if config_str.count('1') != EXPECTED_N_ELEC:
        continue # 電子數不符，跳過

    # 檢查 S_z (N_up - N_down)
    # 計算 "up" 自旋部分 (1up, 2up, 3up)
    n_up = sum(1 for i in range(0, len(config_str), 2) if config_str[i] == '1')  # 1up, 2up, 3up (索引 0, 2, 4...)
    # 計算 "down" 自旋部分 (1down, 2down)
    n_down = sum(1 for i in range(1, len(config_str), 2) if config_str[i] == '1')  # 1down, 2down (索引 1, 3, 5...)
    
    # 計算 S_z * 2 (up部分是+1，down部分是-1)
    current_s_z_times_2 = (n_up * 1) - (n_down * 1)
    
    # 檢查是否符合守恆條件
    if current_s_z_times_2 == EXPECTED_S_Z_TIMES_2:
        conserved_states[config_str] = count

total_conserved_count = sum(conserved_states.values())
conserved_prob = (total_conserved_count / total_samples) * 100
print(f"(總計: {total_conserved_count} / {total_samples} 樣本, 佔 {conserved_prob:.2f}%)")
print("-" * 40)

if not conserved_states:
    print("  (未發現任何符合條件的組態)")
else:
    # 使用 .most_common() 排序並列印
    for config, count in conserved_states.most_common():
        probability = (count / total_samples) * 100
        print(f"  |{config}> : {count:5} (Prob: {probability:5.2f}%)")

# 最終的守恆組態字典，將 "up" 和 "down" 的部分顯示成左邊是 "up" 右邊是 "down"
final_conserved_dict = {}

final_conserved_dict = {}

for config, count in conserved_states.items():
    # 將 config_str 重新排序為 'up' 部分在前，'down' 部分在後
    up_part = ''.join(config[i] for i in range(0, len(config), 2))  # "1up, 2up, 3up"
    down_part = ''.join(config[i] for i in range(1, len(config), 2))  # "1down, 2down"
    
    # 直接將 up_part 和 down_part 作為字典的鍵和值
    final_conserved_dict[(up_part+ down_part)] = count

print("\n--- [守恆組態的字典格式] ---")
print(final_conserved_dict)


(總計: 94 / 2560 樣本, 佔 3.67%)
----------------------------------------
  |001000110100> :     4 (Prob:  0.16%)
  |000110001001> :     4 (Prob:  0.16%)
  |000001100011> :     3 (Prob:  0.12%)
  |000100001011> :     3 (Prob:  0.12%)
  |000100001110> :     3 (Prob:  0.12%)
  |001100000110> :     3 (Prob:  0.12%)
  |001000000111> :     3 (Prob:  0.12%)
  |001100010010> :     2 (Prob:  0.08%)
  |000110010010> :     2 (Prob:  0.08%)
  |000110001100> :     2 (Prob:  0.08%)
  |000100110010> :     2 (Prob:  0.08%)
  |000000110110> :     2 (Prob:  0.08%)
  |000111001000> :     2 (Prob:  0.08%)
  |001100000011> :     2 (Prob:  0.08%)
  |100000011100> :     2 (Prob:  0.08%)
  |000110100001> :     2 (Prob:  0.08%)
  |000010010110> :     2 (Prob:  0.08%)
  |100100100100> :     1 (Prob:  0.04%)
  |000001011010> :     1 (Prob:  0.04%)
  |000000111100> :     1 (Prob:  0.04%)
  |010010000011> :     1 (Prob:  0.04%)
  |100101100000> :     1 (Prob:  0.04%)
  |000001100110> :     1 (Prob:  0.04%)
  |00100100

In [56]:
from qiskit_addon_sqd.counts import BitArray
bitstrings = list(final_conserved_dict.keys())
bitvalues = [int(b, 2) for b in bitstrings]

# 根據出現次數生成對應的數據
samples = []
for bitstring, count in final_conserved_dict.items():
    samples.extend([int(bitstring, 2)] * count)  # 重複出現的數字

# 找出 num_bits
num_bits = len(bitstrings[0])  # 假設所有的 bit 字串長度相同

# 將這些整數數據打包為 uint8 陣列
num_bytes = (num_bits + 7) // 8  # 每個 bit 字串的大小（以字節計算）

# 轉換為二進位字節串
data = b"".join(val.to_bytes(num_bytes, "big") for val in samples)
array = np.frombuffer(data, dtype=np.uint8)

# 假設你已經有 BitArray 類別（如前面提供的代碼）
# 你可以將這個數據轉換為 BitArray 類型
bit_array = BitArray(array.reshape(-1, num_bytes), num_bits=num_bits)

# 輸出結果
print(bit_array)

BitArray(<shape=(), num_shots=94, num_bits=12>)


In [57]:
from qiskit_addon_sqd.counts import BitArray
bitstrings = list(config_counts.keys())
bitvalues = [int(b, 2) for b in bitstrings]

# 根據出現次數生成對應的數據
samples = []
for bitstring, count in config_counts.items():
    samples.extend([int(bitstring, 2)] * count)  # 重複出現的數字

# 找出 num_bits
num_bits = len(bitstrings[0])  # 假設所有的 bit 字串長度相同

# 將這些整數數據打包為 uint8 陣列
num_bytes = (num_bits + 7) // 8  # 每個 bit 字串的大小（以字節計算）

# 轉換為二進位字節串
data = b"".join(val.to_bytes(num_bytes, "big") for val in samples)
array = np.frombuffer(data, dtype=np.uint8)

# 假設你已經有 BitArray 類別（如前面提供的代碼）
# 你可以將這個數據轉換為 BitArray 類型
bit_array = BitArray(array.reshape(-1, num_bytes), num_bits=num_bits)

# 輸出結果
print(bit_array)

BitArray(<shape=(), num_shots=2560, num_bits=12>)


In [58]:
n_elec

(2, 2)

In [59]:
from functools import partial
 
from qiskit_addon_sqd.fermion import (
    SCIResult,
    diagonalize_fermionic_hamiltonian,
    solve_sci_batch,
)
 
# SQD options
energy_tol = 1e-6
occupancies_tol = 1e-6
max_iterations = 5
 
# Eigenstate solver options
num_batches = 3
samples_per_batch = 100
symmetrize_spin = True
carryover_threshold = 1e-4
max_cycle = 200
 
# Pass options to the built-in eigensolver. If you just want to use the defaults,
# you can omit this step, in which case you would not specify the sci_solver argument
# in the call to diagonalize_fermionic_hamiltonian below.
sci_solver = partial(solve_sci_batch, spin_sq=0.0, max_cycle=max_cycle)
 
# List to capture intermediate results
result_history = []
 
 
def callback(results: list[SCIResult]):
    result_history.append(results)
    iteration = len(result_history)
    print(f"Iteration {iteration}")
    for i, result in enumerate(results):
        print(f"\tSubsample {i}")
        print(f"\t\tEnergy: {result.energy + nuclear_repulsion_energy}")
        print(
            f"\t\tSubspace dimension: {np.prod(result.sci_state.amplitudes.shape)}"
        )
 
 
result = diagonalize_fermionic_hamiltonian(
    hcore,
    eri,
    bit_array,
    samples_per_batch=samples_per_batch,
    norb=n_orb,
    nelec=n_elec,
    num_batches=num_batches,
    energy_tol=energy_tol,
    occupancies_tol=occupancies_tol,
    max_iterations=max_iterations,
    sci_solver=sci_solver,
    symmetrize_spin=symmetrize_spin,
    carryover_threshold=carryover_threshold,
    callback=callback,
    seed=12345,
)

Iteration 1
	Subsample 0
		Energy: -7.634167329314557
		Subspace dimension: 225
	Subsample 1
		Energy: -7.634167329314557
		Subspace dimension: 225
	Subsample 2
		Energy: -7.634167329314557
		Subspace dimension: 225
Iteration 2
	Subsample 0
		Energy: -7.634167329314557
		Subspace dimension: 225
	Subsample 1
		Energy: -7.634167329314557
		Subspace dimension: 225
	Subsample 2
		Energy: -7.634167329314557
		Subspace dimension: 225
